In [2]:
import pandas as pd

df=pd.read_pickle("stock_div.pkl")
df.head()

순위    종목코드                종목명 주식종류    시장구분  주당배당금  시가배당율  액면가배당율    액면가 결산월
0  1  084670               동양고속  보통주  유가증권시장  4,700  16.90   94.00  5,000  12
1  2  034950             한국기업평가  보통주   코스닥시장  8,618  14.30  172.36  5,000  12
2  3  172580  하이골드오션12호국제선박투자회사  보통주  유가증권시장      0  13.10    7.32  5,000  12
3  4  095720              웅진씽크빅  보통주  유가증권시장    310  12.60   62.00    500  12
4  5  155900       바다로19호선박투자회사  보통주  유가증권시장      0  11.80    7.00  5,000  12

In [2]:
df.groupby(["시장구분","주식종류"]).size()

시장구분    주식종류
기타비상장   보통주      99
        우선주      63
유가증권시장  보통주     573
        우선주      79
코넥스시장   보통주      11
코스닥시장   보통주     578
프리보드시장  보통주      22
dtype: int64

In [3]:
df=df[df["시장구분"].isin(["유가증권시장","코스닥시장"])]

In [4]:
df.dtypes

순위        object
종목코드      object
종목명       object
주식종류      object
시장구분      object
주당배당금     object
시가배당율     object
액면가배당율    object
액면가       object
결산월       object
dtype: object

In [5]:
df["주당배당금"]=df["주당배당금"].str.replace(",","").astype(int)
df["시가배당율"]=df["시가배당율"].astype(float)

In [45]:
df=df.loc[df["주당배당금"]!=0]

In [30]:
df.loc[df["종목명"]=="한국화장품제조","시가배당율"]=0.32
df.loc[df["종목명"]=="알서포트","시가배당율"]=0.77

In [31]:
df["배당기준주가"]=df["주당배당금"]/df["시가배당율"]*100

In [8]:
df[df["종목명"]=="삼성전자"]

순위    종목코드   종목명 주식종류    시장구분  주당배당금  시가배당율    액면가배당율  액면가 결산월  \
4  305  005930  삼성전자  보통주  유가증권시장   1416    2.9  1,416.00  100  12   

         배당기준주가  
4  48827.586207

In [9]:
import FinanceDataReader as fdr
import time

def get_cur(code):
    time.sleep(0.3)
    try:
        val=fdr.DataReader(code,"20200416")["Close"].mean()
    except:
        val=None
    return val

현재 종가를 받아오는 코드(아래). 시간소요

In [10]:
df["현재시가"]=df["종목코드"].apply(get_cur)

In [17]:
df.head()

순위    종목코드                종목명 주식종류    시장구분  주당배당금  시가배당율  액면가배당율    액면가 결산월  \
0  1  084670               동양고속  보통주  유가증권시장   4700   16.9   94.00  5,000  12   
1  2  034950             한국기업평가  보통주   코스닥시장   8618   14.3  172.36  5,000  12   
2  3  172580  하이골드오션12호국제선박투자회사  보통주  유가증권시장      0   13.1    7.32  5,000  12   
3  4  095720              웅진씽크빅  보통주  유가증권시장    310   12.6   62.00    500  12   
4  5  155900       바다로19호선박투자회사  보통주  유가증권시장      0   11.8    7.00  5,000  12   

         배당기준주가     현재시가       등락률  
0  27810.650888  26700.0 -3.993617  
1  60265.734266  62050.0  2.960664  
2      0.000000   2262.5       inf  
3   2460.317460   2607.5  5.982258  
4      0.000000   2570.0       inf

In [3]:
# df.to_pickle("div.pkl")
df=pd.read_pickle("div.pkl")

In [4]:
df["등락률"]=(df["현재시가"]-df["배당기준주가"])/df["배당기준주가"] * 100

import plotly.express as px
px.scatter(df,x="시가배당율",y="등락률",color="시장구분",hover_name="종목명")